Mathematical Induction Using Matrices
-------------------------------------------------------

In this section, we consider a model for mathematical induction and
investigate how matrices can be used to automatate the machinations of
the process.

# Definition of mathematical induction

Mathematical induction at a minimum consists of two steps: 1.The basis
(base case): a simple or trival case of the problem that can be proven
or solved with minimal complexity. 2.The inductive step: prove that more
complex problems can be reduced to or modelled by the simpler case.

Mathematical induction is important to programming because it serves are
the inspiration for recursive algorithms. Consider a case such as the
Pascal's triangle, which describes the coefficients associated with
terms decreasing in powers of a (and increasing in powers of b) in the
product \$(a+b)^n\$.

It is easy to see that for \$n=0\$, we have \$(a+b)^0=1 = 1\*
(a^0)(b^0)\$. Likewise for \$n=1\$ we have \$(a+b)^1= a + b =
1,(a^1)(b^0)+1,(a^0)(b^1)\$. Finally, a small amount of calculation
shows that:

$$ 
\begin{align}
  && (a+b)^2 \\
= && (a+b)*(a+b) \\
= && (a+b)*a+(a+b)*b \\
= && (a^2+ab)+(ab+b^2) \\
= && 1*a^2b^0 + 2*a^1b^1 + 1*a^0b^2 
\end{align} 
$$

We can store these co-efficients in a matrix of size \$NxN\$ where
\$N=n+1\$. for \$n=2\$ this matrix has the form:

$$ 
\begin{vmatrix}
1 && 0 && 0 \\  
1 && 1 && 0 \\
1 && 2 && 1 
\end{vmatrix} 
$$

At this point we note the pattern that the first and last numbers are
always 1 and the inner numbers are the sums of the pair of numbers in
the corresponding position in the previous iteration. We can express
this using matrix notation as:

$$ 
\begin{vmatrix} 
a_{00} && a_{01} && a_{02} \\  
a_{10} && a_{11} && a_{12} \\
a_{20} && a_{21} && a_{22}
\end{vmatrix} 
$$

with the following rules:

$$ 
\begin{align} 
\forall i . && && a_{i0} = && 1 \\
\forall j . && j>0 \implies && a_{0j} = && 0 \\
\forall i , j . && i>0 \wedge j>0 \implies && a^{ij} = && a_{(i-1)(j-1)} + a_{(i-1)j} 
\end{align} 
$$

It is possible to combine these rules into a matrix expression that
takes as an initial basis a matrix containing all zeros and generates
the next iteration up to the \$N^{th}\$ iteration:

$$ 
\begin{vmatrix}
0 && 0 && 0 \\
1 && 0 && 0 \\
0 && 1 && 0 
\end{vmatrix}
*
\begin{vmatrix} 
a_{00} && a_{01} && a_{02} \\
a_{10} && a_{11} && a_{12} \\
a_{20} && a_{21} && a_{22} 
\end{vmatrix}
*
\begin{vmatrix}
0 && 1 && 0 \\
0 && 1 && 1 \\
0 && 0 && 1 
\end{vmatrix}
+
\begin{vmatrix} 
1 && 0 && 0 \\
1 && 0 && 0 \\
1 && 0 && 0 
\end{vmatrix}
=
\begin{vmatrix}
1 && 0 && 0 \\
1 && a_{00}+a_{01} && a_{01}+a_{02} \\
1 && a_{10}+a_{11} && a_{11}+a_{12} 
\end{vmatrix} 
$$

# Verification in Python

We can verify this in Python


In [1]:
from numpy import matlib 
a = matlib.matrix('0 , 0, 0 ; 0 , 0, 0 ; 0, 0, 0') 
lhs=matlib.matrix('0,0,0; 1,0,0;0,1,0') 
rhs=matlib.matrix('0,1,0;0,1,1;0,0,1') 
k=matlib.matrix('1,0,0;1,0,0;1,0,0')

In [2]:
pascalsTriangle=a; 
for i in range(len(pascalsTriangle)): 
    pascalsTriangle=lhs*pascalsTriangle*rhs+k 
print pascalsTriangle

[[1 0 0]
 [1 1 0]
 [1 2 1]]


Given the equation in this form, it is now possible to use matrix
operations to express the algorithm in a more concise manner. Some
efficiencies are immediately obvious. For example, since a is zero, the
output of the first iteration is simply k. On successive iterations (up
to 2), we raise the power of the multiplication by rhs and lhs on the
leading term, while introducing a new term equal to k. We can therefore
unroll the while loop to look as follows:

In [3]:
compare = (lhs*lhs)*k*(rhs*rhs) + lhs*k*rhs + k 
isSame=(pascalsTriangle==compare) 
assert(isSame.all())

# Compact Notation

The observations above allow us to write the equations for Pascal's
Triangle in the compact notation:

$$ 
\sum_{p=0}^{N-1}{\mathbf{lhs^p};\mathbf{k};\mathbf{rhs^p}}
$$

where

$$ 
\mathbf{lhs}=
\begin{vmatrix}
0 && 0 && \cdots && 0 \\
1 && 0 && \cdots && 0 \\
0 && 1 && \cdots && 0 \\  
\vdots && \vdots && \vdots{} \\
0 && 0 && \cdots && 1
\end{vmatrix}
; \mathbf{k}=
\begin{vmatrix}
1 && 0 && \cdots && 0 \\  
1 && 0 && \cdots && 0 \\  
\vdots && \vdots && \vdots{}
\end{vmatrix}
; \mathbf{rhs}=
\begin{vmatrix}
0 && 1 && 0 &&\cdots && 0 \\  
0 && 1 && 1 &&\cdots && 0 \\  
0 && 0 && 1 &&\cdots && 0 \\  
\vdots && \vdots && \vdots && \vdots && \vdots{} \\
0 && 0 &&\cdots && 1 && 1 \\  
0 && 0 && \cdots && 0 && 1  
\end{vmatrix} 
$$

The matlib module from numpy allows this to be scaled easily in the
python language to directly yield the following implementation:

In [4]:
def PascalsTriangle(N):
    pascalsTriangle=matlib.zeros((N,N), dtype=int)
    if(N>0):
        lhs = matlib.eye(N, k=-1, dtype=int)
        k = matlib.matrix([ [1]+([0]*(N-1))]*N, dtype=int)
        rhs=(matlib.identity(N)+lhs)*matlib.eye(N, k=1, dtype=int)
        for p in range(N):
            pascalsTriangle=pascalsTriangle+((lhs**p)*k*(rhs**p))
    return  pascalsTriangle

for n in range(6):
    print PascalsTriangle(n)


[]
[[ 1.]]
[[ 1.  0.]
 [ 1.  1.]]
[[ 1.  0.  0.]
 [ 1.  1.  0.]
 [ 1.  2.  1.]]
[[ 1.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 1.  2.  1.  0.]
 [ 1.  3.  3.  1.]]
[[ 1.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  2.  1.  0.  0.]
 [ 1.  3.  3.  1.  0.]
 [ 1.  4.  6.  4.  1.]]
